In [11]:
# import torch
# from src.utils.fourier import dft, idft

# t = torch.linspace(0, 2*torch.pi, 1001).reshape(1, -1, 1)
# y = torch.sin(t)

# fft_y = torch.fft.fft(y, dim=1)



torch.Size([1, 1001, 1])
